In [2]:
from Bio.PDB import PDBParser, MMCIFParser, Superimposer  
import numpy as np  
import os
import re
import pandas as pd
from tqdm import tqdm, trange

In [3]:
def align_proteins_and_calculate_rmsd(label_cif, pred_pdb, chain_id):  
    # 创建PDB解析器  
    parser_pdb = PDBParser()
    parser_cif = MMCIFParser()
  
    # 解析两个蛋白质结构
    structure1 = parser_cif.get_structure("protein1", label_cif)  
    structure2 = parser_pdb.get_structure("protein2", pred_pdb)
    
    # 获取Cα原子  
    model1 = structure1[0]
    model2 = structure2[0]
    chain1 = model1[chain_id]
    chain2 = [chain for chain in model2][0]
    # print('chain1:', [res for res in chain1])
    # print('chain2:', [res for res in chain2])
    res_lst1 = [res for res in chain1]
    res_lst2 = [res for res in chain2]
    residues1 = []
    residues2 = []
    ca_atoms1 = []
    ca_atoms2 = []
    for i in range(min(len(res_lst1),len(res_lst2))):
        if (res_lst1[i].get_id()[1] == res_lst2[i].get_id()[1]) & (res_lst1[i].get_resname() == res_lst2[i].get_resname()):
            residues1.append(res_lst1[i])
            residues2.append(res_lst2[i])
            continue
                
    # print('residues1:', residues1, 'residues2:', residues2)
    # if (len(residues1)==0) & (len(res_lst2)<=len(res_lst1)) & (len(res_lst2)>2):
    #     exit=0
    #     for i in range(len(res_lst2)-2):
    #         for j in range(len(res_lst1)-2):
    #             if (res_lst2[i].get_resname()==res_lst1[j].get_resname()) & (res_lst2[i+1].get_resname()==res_lst1[j+1].get_resname()) & (res_lst2[i+2].get_resname()==res_lst1[j+2].get_resname()):
    #                 for k in range(len(res_lst2)-i):
    #                     if res_lst2[i+k].get_resname()==res_lst1[j+k].get_resname():
    #                         residues1.append(res_lst1[j+k])
    #                         residues2.append(res_lst2[i+k])
    #                     else:
    #                         exit=1
    #                         break
    #                     exit=1
    #             if exit==1:
    #                 break
    #         if exit==1:
    #             break
    if (len(residues1)<min(len(res_lst1),len(res_lst2))/4) & (min(len(res_lst1),len(res_lst2))>5):
        exit=0
        for i in range(len(res_lst2)-5):
            for j in range(len(res_lst1)-5):
                if (res_lst2[i].get_resname()==res_lst1[j].get_resname()) & \
                (res_lst2[i+1].get_resname()==res_lst1[j+1].get_resname()) & \
                (res_lst2[i+2].get_resname()==res_lst1[j+2].get_resname()) & \
                (res_lst2[i+3].get_resname()==res_lst1[j+3].get_resname()) & \
                (res_lst2[i+4].get_resname()==res_lst1[j+4].get_resname()) & \
                (res_lst2[i+5].get_resname()==res_lst1[j+5].get_resname()):
                    for k in range(min((len(res_lst2)-i),(len(res_lst1)-j))):
                        if res_lst2[i+k].get_resname()==res_lst1[j+k].get_resname():
                            residues1.append(res_lst1[j+k])
                            residues2.append(res_lst2[i+k])
                        else:
                            exit=1
                            break
                        exit=1
                if exit==1:
                    break
            if exit==1:
                break
                
    for residue1 in residues1:
        for atom1 in residue1:
            if atom1.get_name() == 'CA':
                ca_atoms1.append(atom1)
    for residue2 in residues2:
        for atom2 in residue2:
            if atom2.get_name() == 'CA':
                ca_atoms2.append(atom2)
    # print('ca_atoms1:', len(ca_atoms1), 'ca_atoms2:', len(ca_atoms2))  

    # atoms1 = [atom for atom in structure1.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'C' or atom.get_name() == 'N' or atom.get_name() == 'O']  
    # atoms2 = [atom for atom in structure2.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'C' or atom.get_name() == 'N' or atom.get_name() == 'O']

  
    # 确保两个蛋白质有相同数量的Cα原子  
    # if len(ca_atoms1) != len(ca_atoms2):  
    #     raise ValueError("两个蛋白质必须包含相同数量的Cα原子。") 

    # if len(atoms1) != len(atoms2):  
    #     raise ValueError("两个蛋白质必须包含相同数量的主链原子。")   
  
    # 使用Superimposer进行对齐  
    if (len(ca_atoms1)!=0) & (len(ca_atoms1)==len(ca_atoms2)) :
        superimposer = Superimposer()  
        superimposer.set_atoms(ca_atoms1, ca_atoms2)   
      
        ca_rmsd = superimposer.rms
        superimposer.rotran
        ca_aligned_rmsd = superimposer.rms
    else:
        ca_rmsd = np.nan
        ca_aligned_rmsd = np.nan
 
    # superimposer.set_atoms(atoms1, atoms2)   
  
    # rmsd = superimposer.rms
    # superimposer.rotran
    # aligned_rmsd = superimposer.rms
  
    return ca_rmsd, ca_aligned_rmsd, len(residues1)  

## calcRMSD

In [35]:
filePath = '/home/liuyu/codes/ifold/my_works/output_esmwt/predictions'
pred_lst = os.listdir(filePath)
pred_lst

['5fdu_1y_model_1_unrelaxed.pdb',
 'timings.json',
 '5fdu_1y_model_1_relaxed.pdb',
 '7arh_V_model_1_unrelaxed.pdb',
 '7arh_V_model_1_relaxed.pdb',
 '1lvb_C_model_1_unrelaxed.pdb',
 '1lvb_C_model_1_relaxed.pdb',
 '5gic_C_model_1_unrelaxed.pdb',
 '5gic_C_model_1_relaxed.pdb',
 '3rwi_C_model_1_unrelaxed.pdb',
 '3rwi_C_model_1_relaxed.pdb',
 '2mq5_A_model_1_unrelaxed.pdb',
 '2mq5_A_model_1_relaxed.pdb',
 '6slj_Q_model_1_unrelaxed.pdb',
 '6slj_Q_model_1_relaxed.pdb',
 '5kd4_P_model_1_unrelaxed.pdb',
 '5kd4_P_model_1_relaxed.pdb',
 '5y49_D_model_1_unrelaxed.pdb',
 '5y49_D_model_1_relaxed.pdb',
 '3sp6_B_model_1_unrelaxed.pdb',
 '3sp6_B_model_1_relaxed.pdb',
 '4ds1_B_model_1_unrelaxed.pdb',
 '4ds1_B_model_1_relaxed.pdb',
 '4pra_C_model_1_unrelaxed.pdb',
 '4pra_C_model_1_relaxed.pdb',
 '1yow_B_model_1_unrelaxed.pdb',
 '1yow_B_model_1_relaxed.pdb',
 '1kzz_B_model_1_unrelaxed.pdb',
 '1kzz_B_model_1_relaxed.pdb',
 '2y3y_Q_model_1_unrelaxed.pdb',
 '2y3y_Q_model_1_relaxed.pdb',
 '6wbb_B_model_1_unre

In [36]:
pred_set = []
for item in pred_lst:
    x = re.search('^(.*_.*)_model_1_unrelaxed.pdb', item)
    if x:
        pred_set.append(x.group(1))
pred_set

['5fdu_1y',
 '7arh_V',
 '1lvb_C',
 '5gic_C',
 '3rwi_C',
 '2mq5_A',
 '6slj_Q',
 '5kd4_P',
 '5y49_D',
 '3sp6_B',
 '4ds1_B',
 '4pra_C',
 '1yow_B',
 '1kzz_B',
 '2y3y_Q',
 '6wbb_B',
 '2w9r_B',
 '2mow_B',
 '5t6y_C',
 '6od0_D',
 '2vdo_C',
 '2rsk_C',
 '2jbu_C',
 '3mne_B',
 '1mdi_B',
 '4zqw_A',
 '1e91_B',
 '3g2s_C',
 '1mvu_P',
 '3pkn_B',
 '5ixf_B',
 '4erz_D',
 '5vlh_Y',
 '6kz1_B',
 '2w2u_C',
 '1ssb_B',
 '6o26_C',
 '1k4w_B',
 '6f55_B',
 '5v21_B',
 '6mbj_Y',
 '6ef3_n',
 '1lb7_A',
 '7nw1_CCC',
 '6v2e_B',
 '4xcf_P',
 '4p4x_A',
 '1f4v_D',
 '2ll7_B',
 '6e49_D',
 '4djs_B',
 '6mce_B',
 '1e0q_A',
 '3cfv_E',
 '1ncp_C',
 '2md6_A',
 '2m79_A',
 '5bn6_A',
 '3plu_C',
 '5ykq_A',
 '2lx6_A',
 '6rk9_C',
 '2rqu_B',
 '5wrd_C',
 '1oqp_B',
 '2lqy_A',
 '6zqs_B',
 '1du1_A',
 '2w84_B',
 '3mls_P',
 '6ves_A',
 '5jqf_A',
 '7lc8_A',
 '1j73_A',
 '2jni_A',
 '3u5o_I',
 '4bxl_C',
 '5jtp_E',
 '2bbl_A',
 '3r7g_B',
 '5oa1_P',
 '2lmf_A',
 '2eqh_A',
 '2ak0_A',
 '6oni_D',
 '1vda_A',
 '2m8s_B',
 '2ncw_A',
 '1pd7_B',
 '6oyy_A',
 '2jzi_

In [37]:
len(pred_set)

3665

In [38]:
filePath = '/home/liuyu/codes/ifold/my_works/testset/mmcif_test'
cif_lst = os.listdir(filePath)
cif_lst

['5eec.cif',
 '6v0v.cif',
 '5whq.cif',
 '6khh.cif',
 '3tg4.cif',
 '5lg4.cif',
 '6tdu.cif',
 '3vkw.cif',
 '3f9p.cif',
 '3vop.cif',
 '5drn.cif',
 '5c1t.cif',
 '3oq9.cif',
 '4uo4.cif',
 '7cww.cif',
 '7anm.cif',
 '2enw.cif',
 '6bta.cif',
 '2aqx.cif',
 '4ntd.cif',
 '3kt5.cif',
 '2j6z.cif',
 '6lqk.cif',
 '5svb.cif',
 '1faz.cif',
 '4f6o.cif',
 '6cfw.cif',
 '3ls2.cif',
 '6utl.cif',
 '4bl9.cif',
 '4bv4.cif',
 '6zk9.cif',
 '1j9w.cif',
 '3kkj.cif',
 '2l82.cif',
 '1uzm.cif',
 '3u5o.cif',
 '3ilr.cif',
 '3ru9.cif',
 '4bxl.cif',
 '3myo.cif',
 '3opq.cif',
 '6blb.cif',
 '6ud1.cif',
 '2iy0.cif',
 '5jr1.cif',
 '1plf.cif',
 '7jr5.cif',
 '4a47.cif',
 '1jnv.cif',
 '6nmv.cif',
 '1f0z.cif',
 '2uy2.cif',
 '5zch.cif',
 '5ksb.cif',
 '1fmh.cif',
 '6y9e.cif',
 '6zcf.cif',
 '4qxg.cif',
 '6cnb.cif',
 '1mt1.cif',
 '6niq.cif',
 '7cdc.cif',
 '4r98.cif',
 '2dax.cif',
 '5n0b.cif',
 '2y6s.cif',
 '4yeu.cif',
 '5ej1.cif',
 '2i9n.cif',
 '2qrx.cif',
 '7amc.cif',
 '6nov.cif',
 '4raf.cif',
 '2fca.cif',
 '6dzm.cif',
 '5me5.cif',

In [39]:
len(cif_lst)

4809

In [ ]:
import warnings
warnings.filterwarnings("ignore")
relax_prot_len = []
relax_lst = []
relaxalign_lst = []
unrelax_prot_len = []
unrelax_lst = []
unrelaxalign_lst = []
for item in pred_set:
    prot_id, chain_id = item.split('_')
    label_path = f'/home/liuyu/codes/ifold/my_works/testset/mmcif_test/{prot_id}.cif'
    print(prot_id, chain_id)
    try:
        relaxpred_filename = list(filter(lambda x: re.match(f'{prot_id}_(.*)_.*(_relaxed.pdb)', x) != None, pred_lst))[0]
        relaxpred_path = f'/home/liuyu/codes/ifold/my_works/output_esmwt/predictions/{relaxpred_filename}'
        relax_rmsd, relax_aligned_rmsd, relax_len = align_proteins_and_calculate_rmsd(label_path, relaxpred_path, chain_id)
        # print(relax_rmsd)
        relax_lst.append(relax_rmsd)
        relaxalign_lst.append(relax_aligned_rmsd)
        relax_prot_len.append(relax_len)
    except:
        relax_lst.append(np.nan)
        relaxalign_lst.append(np.nan)
        relax_prot_len.append(np.nan)
    try:
        unrelaxpred_filename = list(filter(lambda x: re.match(f'{prot_id}.*(_unrelaxed.pdb)', x) != None, pred_lst))[0]
        unrelaxpred_path = f'/home/liuyu/codes/ifold/my_works/output_esmwt/predictions/{unrelaxpred_filename}'
        unrelax_rmsd, unrelax_aligned_rmsd, unrelax_len = align_proteins_and_calculate_rmsd(label_path, unrelaxpred_path, chain_id)
        print(unrelax_rmsd)
        unrelax_lst.append(unrelax_rmsd)
        unrelaxalign_lst.append(unrelax_aligned_rmsd)
        unrelax_prot_len.append(unrelax_len)
    except:
        unrelax_lst.append(np.nan)
        unrelaxalign_lst.append(np.nan)
        unrelax_prot_len.append(np.nan)

5fdu 1y
3.010196943180945
7arh V
2.386594692885144
1lvb C
2.0288674169628993
5gic C


In [43]:
df_rst = pd.DataFrame({'prot_id':pred_set, 'prot_lenth':unrelax_prot_len, 'relax_rmsd_esm':relax_lst, 'relaxali_rmsd_esm':relaxalign_lst, 'unrelax_rmsd_esm':unrelax_lst, 'unrelaxali_rmsd_esm':unrelaxalign_lst})

In [44]:
df_rst

,prot_id,prot_lenth,relax_rmsd_esm,relaxali_rmsd_esm,unrelax_rmsd_esm,unrelaxali_rmsd_esm
0,5fdu_1y,10,3.119323,3.119323,3.010197,3.010197
1,7arh_V,10,2.341292,2.341292,2.386595,2.386595
2,1lvb_C,10,2.171236,2.171236,2.028867,2.028867
3,5gic_C,10,1.262418,1.262418,1.267133,1.267133
4,3rwi_C,10,2.256646,2.256646,2.211604,2.211604
...,...,...,...,...,...,...
3660,6g2i_A,166,5.508709,5.508709,2.605178,2.605178
3661,6rw8_A,1319,24.643748,24.643748,25.587509,25.587509
3662,6gjc_A,1710,44.868734,44.868734,44.671536,44.671536
3663,7dxk_A,232,4.031596,4.031596,1.486452,1.486452


In [45]:
df_rst1 = df_rst.dropna()
df_rst1

,prot_id,prot_lenth,relax_rmsd_esm,relaxali_rmsd_esm,unrelax_rmsd_esm,unrelaxali_rmsd_esm
0,5fdu_1y,10,3.119323,3.119323,3.010197,3.010197
1,7arh_V,10,2.341292,2.341292,2.386595,2.386595
2,1lvb_C,10,2.171236,2.171236,2.028867,2.028867
3,5gic_C,10,1.262418,1.262418,1.267133,1.267133
4,3rwi_C,10,2.256646,2.256646,2.211604,2.211604
...,...,...,...,...,...,...
3660,6g2i_A,166,5.508709,5.508709,2.605178,2.605178
3661,6rw8_A,1319,24.643748,24.643748,25.587509,25.587509
3662,6gjc_A,1710,44.868734,44.868734,44.671536,44.671536
3663,7dxk_A,232,4.031596,4.031596,1.486452,1.486452


In [46]:
df_rst2 = df_rst1[df_rst1['prot_lenth']>=10]
df_rst2

,prot_id,prot_lenth,relax_rmsd_esm,relaxali_rmsd_esm,unrelax_rmsd_esm,unrelaxali_rmsd_esm
0,5fdu_1y,10,3.119323,3.119323,3.010197,3.010197
1,7arh_V,10,2.341292,2.341292,2.386595,2.386595
2,1lvb_C,10,2.171236,2.171236,2.028867,2.028867
3,5gic_C,10,1.262418,1.262418,1.267133,1.267133
4,3rwi_C,10,2.256646,2.256646,2.211604,2.211604
...,...,...,...,...,...,...
3660,6g2i_A,166,5.508709,5.508709,2.605178,2.605178
3661,6rw8_A,1319,24.643748,24.643748,25.587509,25.587509
3662,6gjc_A,1710,44.868734,44.868734,44.671536,44.671536
3663,7dxk_A,232,4.031596,4.031596,1.486452,1.486452


In [47]:
df_rst2.to_csv('rmsd_result_esmwt_10.csv', index=False)

## calcRMSD_new

In [3]:
filePath = '/home/liuyu/codes/ifold/my_works/output_esmwt5_250409/predictions'
pred_lst = os.listdir(filePath)
pred_lst

['8vs6_E_model_1_unrelaxed.pdb',
 'timings.json',
 '8vs6_E_model_1_relaxed.pdb',
 '8j47_D_model_1_unrelaxed.pdb',
 '8j47_D_model_1_relaxed.pdb',
 '8ihu_A_model_1_unrelaxed.pdb',
 '8ihu_A_model_1_relaxed.pdb',
 '8jxg_D_model_1_unrelaxed.pdb',
 '8jxg_D_model_1_relaxed.pdb',
 '8r7o_A_model_1_unrelaxed.pdb',
 '8r7o_A_model_1_relaxed.pdb',
 '8cn6_A_model_1_unrelaxed.pdb',
 '8cn6_A_model_1_relaxed.pdb',
 '8eph_A_model_1_unrelaxed.pdb',
 '8eph_A_model_1_relaxed.pdb',
 '8fpq_A_model_1_unrelaxed.pdb',
 '8fpq_A_model_1_relaxed.pdb',
 '7hg0_A_model_1_unrelaxed.pdb',
 '7hg0_A_model_1_relaxed.pdb',
 '8vdg_A_model_1_unrelaxed.pdb',
 '8vdg_A_model_1_relaxed.pdb',
 '9imp_A_model_1_unrelaxed.pdb',
 '9imp_A_model_1_relaxed.pdb',
 '8v25_A_model_1_unrelaxed.pdb',
 '8v25_A_model_1_relaxed.pdb',
 '8wke_A_model_1_unrelaxed.pdb',
 '8wke_A_model_1_relaxed.pdb',
 '9f2g_A_model_1_unrelaxed.pdb',
 '9f2g_A_model_1_relaxed.pdb',
 '7ga8_A_model_1_unrelaxed.pdb',
 '7ga8_A_model_1_relaxed.pdb',
 '8v4f_B_model_1_unrela

In [4]:
pred_set = []
for item in pred_lst:
    x = re.search('^(.*_.*)_model_1_unrelaxed.pdb', item)
    if x:
        pred_set.append(x.group(1))
pred_set

['8vs6_E',
 '8j47_D',
 '8ihu_A',
 '8jxg_D',
 '8r7o_A',
 '8cn6_A',
 '8eph_A',
 '8fpq_A',
 '7hg0_A',
 '8vdg_A',
 '9imp_A',
 '8v25_A',
 '8wke_A',
 '9f2g_A',
 '7ga8_A',
 '8v4f_B',
 '8ve1_A',
 '7gus_A',
 '8uax_A',
 '9jwd_A',
 '8uxv_N',
 '8vif_H',
 '9evy_A',
 '8rc6_A',
 '9f5j_A',
 '8r74_A',
 '8jrn_D',
 '8jfy_A',
 '8qvy_A',
 '9ild_A',
 '8k33_A',
 '8qhs_A',
 '8hx0_A',
 '8bwd_A',
 '8r7w_A',
 '8cpr_A',
 '9k8m_A',
 '8khr_C',
 '9fzd_A',
 '8qog_A',
 '8fpx_A',
 '8vsj_A',
 '8k4u_A',
 '8qwk_A',
 '9gnl_A',
 '9egr_A',
 '8c0y_A',
 '8xk2_A',
 '8k4q_A',
 '9drd_A',
 '8jji_B',
 '8kdr_L',
 '8xx0_L',
 '8v4i_L',
 '9dn4_A',
 '8ikg_A',
 '8iyv_A',
 '8v8s_A',
 '8v91_A',
 '8xni_A',
 '8kdu_A',
 '8wfm_A',
 '8r0z_A',
 '8bze_A',
 '8r3w_A',
 '8r40_R',
 '8hyc_A',
 '8ji9_C',
 '8gl7_A',
 '8k1b_A',
 '8cbk_A',
 '8kex_E',
 '8qqa_AAA',
 '8r2f_AAA',
 '7mu6_A',
 '8pc9_A',
 '8r2y_A',
 '8jxo_A',
 '8joi_A',
 '8p2t_H',
 '8bm8_A',
 '8jxs_A',
 '7szw_A',
 '8f7m_A',
 '8ghl_A',
 '8qu7_E',
 '8jef_A',
 '8jjp_A',
 '8qt0_A',
 '8k3z_A',
 '8qqe

In [5]:
len(pred_set)

171

In [6]:
filePath = '/home/liuyu/codes/ifold/my_works/testset/mmcif_test_250409'
cif_lst = os.listdir(filePath)
cif_lst

['7exa.cif',
 '7g2h.cif',
 '7ga8.cif',
 '7gus.cif',
 '7gy4.cif',
 '7hg0.cif',
 '7mu6.cif',
 '7szw.cif',
 '8bm8.cif',
 '8bop.cif',
 '8bwd.cif',
 '8bze.cif',
 '8c0y.cif',
 '8c61.cif',
 '8cbk.cif',
 '8cmt.cif',
 '8cn6.cif',
 '8cnk.cif',
 '8cpr.cif',
 '8e6v.cif',
 '8eph.cif',
 '8exx.cif',
 '8f7m.cif',
 '8fpq.cif',
 '8fpx.cif',
 '8ghk.cif',
 '8ghl.cif',
 '8gl7.cif',
 '8hat.cif',
 '8hau.cif',
 '8hqs.cif',
 '8hx0.cif',
 '8hyc.cif',
 '8i3v.cif',
 '8i96.cif',
 '8iar.cif',
 '8ida.cif',
 '8ihu.cif',
 '8ikg.cif',
 '8irj.cif',
 '8iyq.cif',
 '8iyv.cif',
 '8izq.cif',
 '8j47.cif',
 '8jef.cif',
 '8jev.cif',
 '8jfy.cif',
 '8jgm.cif',
 '8ji9.cif',
 '8jji.cif',
 '8jjp.cif',
 '8jkh.cif',
 '8jn8.cif',
 '8jnz.cif',
 '8joi.cif',
 '8jr8.cif',
 '8jrn.cif',
 '8jsi.cif',
 '8jtj.cif',
 '8jwh.cif',
 '8jxd.cif',
 '8jxg.cif',
 '8jxo.cif',
 '8jxs.cif',
 '8jyl.cif',
 '8jzl.cif',
 '8k04.cif',
 '8k0a.cif',
 '8k0f.cif',
 '8k0x.cif',
 '8k10.cif',
 '8k1a.cif',
 '8k1b.cif',
 '8k1m.cif',
 '8k25.cif',
 '8k29.cif',
 '8k33.cif',

In [7]:
len(cif_lst)

191

In [8]:
import warnings
warnings.filterwarnings("ignore")
relax_prot_len = []
relax_lst = []
relaxalign_lst = []
unrelax_prot_len = []
unrelax_lst = []
unrelaxalign_lst = []
for item in pred_set:
    prot_id, chain_id = item.split('_')
    label_path = f'/home/liuyu/codes/ifold/my_works/testset/mmcif_test_250409/{prot_id.lower()}.cif'
    print(prot_id, chain_id)
    try:
        relaxpred_filename = list(filter(lambda x: re.match(f'{prot_id}_{chain_id}_(.*)_.*(_relaxed.pdb)', x) != None, pred_lst))[0]
        relaxpred_path = f'/home/liuyu/codes/ifold/my_works/output_esmwt5_250409/predictions/{relaxpred_filename}'
        relax_rmsd, relax_aligned_rmsd, relax_len = align_proteins_and_calculate_rmsd(label_path, relaxpred_path, chain_id)
        # print(relax_rmsd)
        relax_lst.append(relax_rmsd)
        relaxalign_lst.append(relax_aligned_rmsd)
        relax_prot_len.append(relax_len)
    except:
        relax_lst.append(np.nan)
        relaxalign_lst.append(np.nan)
        relax_prot_len.append(np.nan)
    try:
        unrelaxpred_filename = list(filter(lambda x: re.match(f'{prot_id}_{chain_id}.*(_unrelaxed.pdb)', x) != None, pred_lst))[0]
        unrelaxpred_path = f'/home/liuyu/codes/ifold/my_works/output_esmwt5_250409/predictions/{unrelaxpred_filename}'
        unrelax_rmsd, unrelax_aligned_rmsd, unrelax_len = align_proteins_and_calculate_rmsd(label_path, unrelaxpred_path, chain_id)
        print(unrelax_rmsd)
        unrelax_lst.append(unrelax_rmsd)
        unrelaxalign_lst.append(unrelax_aligned_rmsd)
        unrelax_prot_len.append(unrelax_len)
    except:
        unrelax_lst.append(np.nan)
        unrelaxalign_lst.append(np.nan)
        unrelax_prot_len.append(np.nan)

8vs6 E
5.3589392912611205
8j47 D
8.880130038933288
8ihu A
17.8417812504382
8jxg D
1.384686823878283
8r7o A
2.8267385957878055
8cn6 A
1.0131520753772663
8eph A
6.651385272058748
8fpq A
2.246219522359869
7hg0 A
2.2907384702741527
8vdg A
2.792438948286669
9imp A
3.706329373135594
8v25 A
1.6927711902761895
8wke A
1.2700543161953235
9f2g A
0.9467264079832505
7ga8 A
1.2926677630632366
8v4f B
5.1069405255329325
8ve1 A
0.9393765344498486
7gus A
1.068920787348043
8uax A
0.7558939016390691
9jwd A
0.7643466389049587
8uxv N
4.286274906123074
8vif H
3.8084295292611707
9evy A
1.8186470109772341
8rc6 A
3.415355179219296
9f5j A
4.202690338137623
8r74 A
0.8230987222164651
8jrn D
0.891074019735583
8jfy A
4.085718517093803
8qvy A
0.8428274724489357
9ild A
3.2653446993390465
8k33 A
0.6532533542752544
8qhs A
9.856294116758257
8hx0 A
3.236452023095803
8bwd A
21.224721458342792
8r7w A
0.9022765354556662
8cpr A
7.495402940263033
9k8m A
1.4145406117860388
8khr C
24.7956491486055
9fzd A
7.213003655673692
8qog A

In [9]:
df_rst = pd.DataFrame({'prot_id':pred_set, 'prot_lenth':unrelax_prot_len, 'relax_rmsd_esm5':relax_lst, 'relaxali_rmsd_esm5':relaxalign_lst, 'unrelax_rmsd_esm5':unrelax_lst, 'unrelaxali_rmsd_esm5':unrelaxalign_lst})

In [10]:
df_rst

,prot_id,prot_lenth,relax_rmsd_esm5,relaxali_rmsd_esm5,unrelax_rmsd_esm5,unrelaxali_rmsd_esm5
0,8vs6_E,10,5.602477,5.602477,5.358939,5.358939
1,8j47_D,29,8.940256,8.940256,8.880130,8.880130
2,8ihu_A,72,17.864061,17.864061,17.841781,17.841781
3,8jxg_D,72,1.372119,1.372119,1.384687,1.384687
4,8r7o_A,75,2.851029,2.851029,2.826739,2.826739
...,...,...,...,...,...,...
166,8izq_A,996,NaN,NaN,37.953307,37.953307
167,8jtj_A,1175,19.376926,19.376926,19.201443,19.201443
168,8exx_A,1088,13.148237,13.148237,13.117967,13.117967
169,8iyq_A,717,14.615972,14.615972,14.151210,14.151210


In [11]:
df_rst1 = df_rst.dropna()
df_rst1

,prot_id,prot_lenth,relax_rmsd_esm5,relaxali_rmsd_esm5,unrelax_rmsd_esm5,unrelaxali_rmsd_esm5
0,8vs6_E,10,5.602477,5.602477,5.358939,5.358939
1,8j47_D,29,8.940256,8.940256,8.880130,8.880130
2,8ihu_A,72,17.864061,17.864061,17.841781,17.841781
3,8jxg_D,72,1.372119,1.372119,1.384687,1.384687
4,8r7o_A,75,2.851029,2.851029,2.826739,2.826739
...,...,...,...,...,...,...
165,8qmp_A,913,4.143879,4.143879,4.179290,4.179290
167,8jtj_A,1175,19.376926,19.376926,19.201443,19.201443
168,8exx_A,1088,13.148237,13.148237,13.117967,13.117967
169,8iyq_A,717,14.615972,14.615972,14.151210,14.151210


In [12]:
df_rst2 = df_rst1[df_rst1['prot_lenth']>=10]
df_rst2

,prot_id,prot_lenth,relax_rmsd_esm5,relaxali_rmsd_esm5,unrelax_rmsd_esm5,unrelaxali_rmsd_esm5
0,8vs6_E,10,5.602477,5.602477,5.358939,5.358939
1,8j47_D,29,8.940256,8.940256,8.880130,8.880130
2,8ihu_A,72,17.864061,17.864061,17.841781,17.841781
3,8jxg_D,72,1.372119,1.372119,1.384687,1.384687
4,8r7o_A,75,2.851029,2.851029,2.826739,2.826739
...,...,...,...,...,...,...
165,8qmp_A,913,4.143879,4.143879,4.179290,4.179290
167,8jtj_A,1175,19.376926,19.376926,19.201443,19.201443
168,8exx_A,1088,13.148237,13.148237,13.117967,13.117967
169,8iyq_A,717,14.615972,14.615972,14.151210,14.151210


In [13]:
df_rst2.to_csv('rmsd_result_esmwt5_250409.csv', index=False)

In [14]:
df_rst2.describe()

,prot_lenth,relax_rmsd_esm5,relaxali_rmsd_esm5,unrelax_rmsd_esm5,unrelaxali_rmsd_esm5
count,169.000000,169.000000,169.000000,169.000000,169.000000
mean,340.183432,6.447745,6.447745,6.350465,6.350465
std,225.876090,8.149439,8.149439,7.939025,7.939025
min,10.000000,0.564370,0.564370,0.653253,0.653253
25%,179.000000,1.389488,1.389488,1.407849,1.407849
50%,276.000000,2.851029,2.851029,2.801208,2.801208
75%,442.000000,7.242070,7.242070,7.213004,7.213004
max,1340.000000,43.567725,43.567725,43.890834,43.890834
